# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686498


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:05<00:51,  1.90s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:51,  1.98s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:36,  1.46s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:25,  1.06s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  4.33it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  4.36it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  3.99it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.47it/s]

cschwefl                               0.000125
CThomas                                0.000276
Dynamatt                               0.000092
not-logged-in-ff40b895d06514783cbc     0.000959
ElisabethB                             0.000290
rnevils                                0.000096
justsomenoodles                        0.000093
jmfranci                               0.000432
VHOLIK                                 0.000698
not-logged-in-4162114e3c20a1fc0e8b    12.618829
not-logged-in-66748d0532f903be0737     0.007210
DrPZ                                   0.000232
DanielSchmidt                          0.017035
MeepTastic                             0.000352
not-logged-in-75e0b81dd6080af33b88     0.001714
AdamMorse                              0.000145
TheEpicPrimius123                      0.001723
AlexYoung35                            0.000104
RaulE                                  0.002590
alexmnelson18                          0.000119
Lavadude                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())